# Generate CMA using daily data service - F#

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *cumulative moving average - CMA* from a daily data service.

### Inputs/outputs
Cumulative moving averages' requests require instrument's identifier, date time intervals and number of periods as per inputs.

CMA is the average of all the data points.

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve daily data from the hosted service. The queried endpoint in this script are:
* *DailyCmaService*: to directly retrieve daily cma data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.DailyAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily CMA sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.36.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.DailyAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data
To request *daily cma* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates

#### 3.1 Instrument selection

In [ ]:
// Set instrument
let ticker = "AAPL"
let exchange = "XNGS"

#### 3.2 Time period selection

In [ ]:
// Set data interval (we are using Google date time format)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = 2019, Month = 01, Day = 01 ),
    EndDate = new Date ( Year = 2020, Month = 12, Day = 31 )
  )

// Generate constraints based on the previous date selection
// Time constraints are not neeeded for the splits
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)

#### 3.3 CMA request creation
The following code snippet creates *gRPC client*, process daily CMA requests and ensure that the reply is not empty:

In [ ]:
// Instantiate the daily CMA service
let service = new DailyCmaService.DailyCmaServiceClient(channel)

In [ ]:
// Create and process the daily long CMA request
let request = 
    new DailyCmaRequest( 
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker ),
        Constraints = constraints 
    )
let reply = service.DailyCma(request, headers)

// Visualize requests' results
//reply.Data |> display

### Step 4: Visualize data

#### 4.1 Select data

In [ ]:
// Extract the price and the volume
let prices  = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Value)))
let cmaData = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Average)))

#### 4.2 Plot CMA and daily prices
Plot CMA request results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. 

In [ ]:

[
    new Scattergl(
        name = "CMA",
        mode = "lines",
        x = (cmaData |> Seq.map fst),
        y = (cmaData |> Seq.map snd)
    )
    new Scattergl(
        name = "Price",
        mode = "lines",
        x = (prices |> Seq.map fst),
        y = (prices |> Seq.map snd)
    )
]
|> Chart.Plot
|> Chart.WithTitle $"CMA and daily prices for {ticker}"
|> Chart.WithXTitle "Date"
|> Chart.WithWidth  1500
|> Chart.WithHeight 800
|> display